#### Start the docker container for development first

* Go to the root directory of this repository and run  ``docker run -it --rm -v `pwd`:/gws -p 18000:80 gplates/gws /bin/bash``

* Inside the virtual machine: `cd /gws/django/GWS && python3 manage.py runserver 0.0.0.0:80`

* The dev server is at http://localhost:18000)

In [5]:
import requests, json, time, random
SERVER_URL = 'http://127.0.0.1:18000/'
proxies = {'http':''}

points_str=''
for i in range(1000):
    points_str+=f'{random.uniform(-180,180):3.2f},'
    points_str+=f'{random.uniform(-90,90):3.2f},'

#print(points_str)

data={'time':140, 'return_null_points':''}
data['points'] = points_str[:-1]

#t = time.process_time()
%timeit r = requests.post(SERVER_URL+'reconstruct/reconstruct_points/',data=data,verify=False, proxies=proxies)
#elapsed_time = time.process_time() - t

#print(elapsed_time)
#print(r.text)
#print(json.dumps(json.loads(str(r.text)), sort_keys=True, indent=4))



1.21 s ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
data = json.loads(str(r.text))
#print(data)
for i in data['coordinates']:
    print(i)

In [13]:
import requests, json, time, random

def get_lat_lon_at_age(lats, lons, age, model='SETON2012', pids=None, pid=None, 
                       server_url='https://gws.gplates.org/', proxies={'http':''}):
    
    assert len(lats) == len(lons)
    if pids:
        assert len(pids) == len(lons)
        
    data={'time':age, 'return_null_points':''}
    
    
    #construct the request string for points
    points_str=''
    for lat, lon in zip(lats, lons):
        points_str+=f'{lon:3.2f},'
        points_str+=f'{lat:3.2f},'
    points_str= points_str[:-1]
    
    data['points'] = points_str
    
    #construct the request string for plates ids
    if pids:
        pids_str=''
        for pid_ in pids: 
            pids_str+=f'{int(pid_)},'
        
        data['pids'] = pids_str[:-1] 
        #print(data['pids'])
     
    if pid:
        data['pid'] = f'{int(pid)}'
    
    data['model'] = model 
    
    ret = requests.post(server_url+'reconstruct/reconstruct_points/',data=data,verify=False, proxies=proxies)
    #print(ret.text)
    
    data = json.loads(str(ret.text))
    #print(data)
    lats_r=[]
    lons_r=[]
    for c in data['coordinates']:
        if not c:
            lats_r.append(float("NAN"))
            lons_r.append(float("NAN"))
        else:
            lats_r.append(c[1])
            lons_r.append(c[0])
    
    return lats_r, lons_r
 
lats=[]
lons=[]
pids=[]
for i in range(10000):
    lons.append(random.uniform(-180,180))
    lats.append(random.uniform(-90,90))
    pids.append(701)
    
#%timeit lats_r, lons_r = get_lat_lon_at_age(lats, lons, 10, pids=pids, server_url='http://127.0.0.1:18000/') 
lats_r, lons_r = get_lat_lon_at_age(lats, lons, 10, pids=pids, server_url='http://127.0.0.1:18000/')

#%timeit lats_r, lons_r = get_lat_lon_at_age(lats, lons, 10, server_url='http://127.0.0.1:18000/')  
#lats_r, lons_r = get_lat_lon_at_age(lats, lons, 10, server_url='http://127.0.0.1:18000/')  

#%timeit lats_r, lons_r = get_lat_lon_at_age(lats, lons, 10, pid=701, server_url='http://127.0.0.1:18000/')  
#lats_r, lons_r = get_lat_lon_at_age(lats, lons, 10, pid=701, server_url='http://127.0.0.1:18000/')  

print(len(list(zip(lats_r, lons_r))))



10000
